In [13]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from numpy import genfromtxt
from sklearn import kernel_ridge
import os
print(len(os.sched_getaffinity(0)))

2


In [14]:
path_to_train = '/content/wine_training.csv'
path_to_test = '/content/wine_test.csv'

train = genfromtxt(path_to_train, delimiter=',')
test = genfromtxt(path_to_test, delimiter=',')

print('TRAIN SET')
print(train)
print('ALL BUT LAST')
print(train[:, :-1])
print('LAST')
print(train[:, -1])

TRAIN SET
[[ 6.1   0.6   0.08 ...  0.54 11.    5.  ]
 [10.3   0.53  0.48 ...  0.59  9.3   6.  ]
 [ 6.9   0.49  0.19 ...  0.64  9.8   6.  ]
 ...
 [ 7.    0.38  0.49 ...  0.77 11.4   6.  ]
 [10.    0.35  0.47 ...  0.52 12.    6.  ]
 [ 7.    0.6   0.3  ...  1.17 10.2   5.  ]]
ALL BUT LAST
[[ 6.1   0.6   0.08 ...  3.38  0.54 11.  ]
 [10.3   0.53  0.48 ...  3.12  0.59  9.3 ]
 [ 6.9   0.49  0.19 ...  3.38  0.64  9.8 ]
 ...
 [ 7.    0.38  0.49 ...  3.39  0.77 11.4 ]
 [10.    0.35  0.47 ...  3.23  0.52 12.  ]
 [ 7.    0.6   0.3  ...  3.3   1.17 10.2 ]]
LAST
[5. 6. 6. ... 6. 6. 5.]


In [15]:
norm_train = (train - train.mean(axis=0)) / train.std(axis=0)
norm_test = (test - test.mean(axis=0)) / test.std(axis=0)

In [16]:
#Use linear regression to fit a linear model to the training set 
LinearRegressor = sklearn.linear_model.LinearRegression()
LinearRegressor.fit(norm_train[:, :-1], norm_train[:, -1])
predicted_train = LinearRegressor.predict(norm_train[:, :-1])
print(f'LINEAR REGRESSION TRAINING MSE: {sklearn.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
predicted_test = LinearRegressor.predict(norm_test[:, :-1])
print(f'LINEAR REGRESSION TEST MSE: {sklearn.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554817
LINEAR REGRESSION TEST MSE: 0.7021527395264012


In [29]:
#Use kernel ridge regression to fit a non-linear model to the training set

#Gaussian (RBF) Kernel 
def GaussianError(args):
  RBFRegresor = kernel_ridge.KernelRidge(alpha = args[0], kernel = 'rbf', gamma = args[1])
  RBFRegresor.fit(norm_train[:, :-1], norm_train[:, -1])
  predicted_train = RBFRegresor.predict(norm_train[:, :-1])
  #print(f'GAUSSIAN KERNEL TRAINING MSE: {sklearn.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
  predicted_test = RBFRegresor.predict(norm_test[:, :-1])
  #print(f'GAUSSIAN KERNEL TEST MSE: {sklearn.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')
  train_mse = sklearn.metrics.mean_squared_error(predicted_train, norm_train[:, -1])
  test_mse = sklearn.metrics.mean_squared_error(predicted_test, norm_test[:, -1])
  return test_mse

#Laplacian kernel
def LaplacianError(args):
  LaplacianRegresor = kernel_ridge.KernelRidge(alpha = args[0], kernel = 'laplacian', gamma = args[1])
  LaplacianRegresor.fit(norm_train[:, :-1], norm_train[:, -1])
  predicted_train = LaplacianRegresor.predict(norm_train[:, :-1])
  #print(f'LAPLACIAN KERNEL TRAINING MSE: {sklearn.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
  predicted_test = LaplacianRegresor.predict(norm_test[:, :-1])
  #print(f'LAPLACIAN KERNEL TEST MSE: {sklearn.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')
  train_mse = sklearn.metrics.mean_squared_error(predicted_train, norm_train[:, -1])
  test_mse = sklearn.metrics.mean_squared_error(predicted_test, norm_test[:, -1])
  return test_mse

In [18]:
print(GaussianError((0, 1.0)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:197: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


0.7185854801749969


In [33]:
#Best params for RBF
from concurrent.futures import ThreadPoolExecutor
import itertools
executor = ThreadPoolExecutor(len(os.sched_getaffinity(0)))
alpha_range = np.arange(0.9, 1, 0.01)
gamma_range = np.arange(0.1, 0.2, 0.01)
all_combinations = itertools.product(alpha_range, gamma_range)
copy_combinations = list(all_combinations)
results = executor.map(GaussianError, copy_combinations)
copy_results = list(results)
best_val = min(copy_results)
best_index = copy_results.index(best_val)
best_params = copy_combinations[best_index]
print(best_val)
print(best_params)

0.6187216397172022
(0.9900000000000001, 0.12)


In [34]:
#Best params of Laplacian 
from concurrent.futures import ThreadPoolExecutor
import itertools
executor = ThreadPoolExecutor(len(os.sched_getaffinity(0)))
alpha_range = np.arange(0.3, 0.4, 0.01)
gamma_range = np.arange(0.3, 0.4, 0.01)
all_combinations = itertools.product(alpha_range, gamma_range)
copy_combinations = list(all_combinations)
results = executor.map(LaplacianError, copy_combinations)
copy_results = list(results)
best_val = min(copy_results)
best_index = copy_results.index(best_val)
best_params = copy_combinations[best_index]
print(best_val)
print(best_params)

0.586525780237679
(0.3, 0.3)


In [ ]:
#Use 10-fold CV to tune the length of scale sigma and lambda for both above kernels. 

In [ ]:
#MSE of all 3 modesl 

In [ ]:
#Predict on new batch of wines 